In many applications, data may be spread across a number of files or databases or be
arranged in a form that is not easy to analyze. This chapter focuses on tools to help
combine, join, and rearrange data.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 8.1 Hierarchical Indexing

Hierarchical indexing is an important feature of pandas that enables you to have multiple (two or more) index levels on an axis. 

In [32]:
data = pd.Series(data=np.random.randn(9), index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd']
                                                 , [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [33]:
data

a  1    0.588222
   2    0.690070
   3    0.775582
b  1    0.642785
   3    0.737432
c  1   -1.035841
   2    1.448188
d  2    0.680291
   3    1.417015
dtype: float64

What you’re seeing is a prettified view of a Series with a MultiIndex as its index. The
“gaps” in the index display mean “use the label directly above”

In [34]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

With a hierarchically indexed object, so-called partial indexing is possible, enabling
you to concisely select subsets of the data

In [35]:
data

a  1    0.588222
   2    0.690070
   3    0.775582
b  1    0.642785
   3    0.737432
c  1   -1.035841
   2    1.448188
d  2    0.680291
   3    1.417015
dtype: float64

In [36]:
data['b':'d']

b  1    0.642785
   3    0.737432
c  1   -1.035841
   2    1.448188
d  2    0.680291
   3    1.417015
dtype: float64

In [37]:
data.loc[['b', 'd']]

b  1    0.642785
   3    0.737432
d  2    0.680291
   3    1.417015
dtype: float64

In [38]:
data.loc[:, 2]

a    0.690070
c    1.448188
d    0.680291
dtype: float64

In [39]:
data.loc[:,3]

a    0.775582
b    0.737432
d    1.417015
dtype: float64

In [40]:
data.loc['b', 3]

0.7374316861301699

In [41]:
data.unstack() #Convert multi index series into df

,1,2,3
a,0.588222,0.690070,0.775582
b,0.642785,NaN,0.737432
c,-1.035841,1.448188,NaN
d,NaN,0.680291,1.417015


In [42]:
data.unstack().stack() #stack reverses the unstack or convert df into multi index series

a  1    0.588222
   2    0.690070
   3    0.775582
b  1    0.642785
   3    0.737432
c  1   -1.035841
   2    1.448188
d  2    0.680291
   3    1.417015
dtype: float64

With a DataFrame, either axis can have a hierarchical index

In [48]:
frame = pd.DataFrame(data=np.arange(12).reshape(4,3), index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])

In [49]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [56]:
frame.index.names = ['key1', 'key2']

In [57]:
frame.columns.names = ['state', 'color']

In [58]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [59]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [60]:
frame['Colorado']

color      Green
key1 key2       
a    1         2
     2         5
b    1         8
     2        11

## Reordering and Sorting Levels

In [61]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [62]:
frame.swaplevel('key1', 'key2') #.swaplevel() swaps the level of indexes

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [65]:
frame.sort_index(level=1) #.sort_index() sorts the index by taking the level argument

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [68]:
frame.swaplevel(1, 0).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## Summary Statistics by Level

In [69]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [70]:
frame.sum(level='key1')

state  Ohio     Colorado
color Green Red    Green
key1                    
a         3   5        7
b        15  17       19

In [71]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [75]:
frame.sum(level='state', axis=1)

state      Ohio  Colorado
key1 key2                
a    1        1         2
     2        7         5
b    1       13         8
     2       19        11

In [78]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## Indexing with a DataFrame’s columns

In [81]:
df = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1), 'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})

In [82]:
df

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [86]:
df2 = df.set_index(['c', 'd'])

In [87]:
df2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [90]:
df

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [93]:
df.set_index(['c', 'd'], drop=False) #sets the passed column(s) as index, might hierarchical or not

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [96]:
df2.reset_index() #inverse of set_index(), reset the index to int

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# 8.2 Combining and Merging Datasets

Data contained in pandas objects can be combined together in a number of ways:
1. **pandas.merge** connects rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases, as it implements database join operations.
2. **pandas.concat** concatenates or “stacks” together objects along an axis.
3. The **combine_first** instance method enables splicing together overlapping data to fill in missing values in one object with values from another.

## Database-Style DataFrame Joins

In [98]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})

In [99]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})

In [100]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [101]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [103]:
df1.merge(df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [104]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [105]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})

In [106]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})

In [107]:
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [108]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [109]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [117]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey', how='outer')[['lkey', 'rkey', 'data1', 'data2']]

,lkey,rkey,data1,data2
0,b,b,0.0,1.0
1,b,b,1.0,1.0
2,b,b,6.0,1.0
3,a,a,2.0,0.0
4,a,a,4.0,0.0
5,a,a,5.0,0.0
6,c,NaN,3.0,NaN
7,NaN,d,NaN,2.0


*See Table 8-1. Different join types with how argument*

In [118]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})

In [123]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'], 'data2': range(5)})

In [124]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [125]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [126]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [127]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'], 'key2': ['one', 'two', 'one'], 'lval': [1, 2, 3]})

In [128]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'], 'key2': ['one', 'one', 'one', 'two'], 'rval': [4, 5, 6, 7]})

In [129]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [130]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [132]:
pd.merge(right, left, on=['key1', 'key2'])

,key1,key2,rval,lval
0,foo,one,4,1
1,foo,one,5,1
2,bar,one,6,3


In [134]:
pd.merge(right, left, on=['key1', 'key2'], how='outer')

,key1,key2,rval,lval
0,foo,one,4.0,1.0
1,foo,one,5.0,1.0
2,bar,one,6.0,3.0
3,bar,two,7.0,NaN
4,foo,two,NaN,2.0


In [137]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [143]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [160]:
pd.merge(right, left, on=['key1', 'key2'], suffixes=('_right', '_left'), indicator=True, how='outer')

,key1,key2,rval,lval,_merge
0,foo,one,4.0,1.0,both
1,foo,one,5.0,1.0,both
2,bar,one,6.0,3.0,both
3,bar,two,7.0,NaN,left_only
4,foo,two,NaN,2.0,right_only


*See Table 8-2. merge function arguments*

## Merging on Index

In [161]:
left1 = pd.DataFrame({'key1': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})

In [162]:
right1 = pd.DataFrame({'group_vale': [3.5, 7]}, index=['a', 'b'])

In [164]:
left1

,key1,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [167]:
right1

,group_vale
a,3.5
b,7.0


In [168]:
pd.merge(left1, right1, left_on='key1', right_index=True)

,key1,value,group_vale
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [170]:
pd.merge(left1, right1, left_on='key1', right_index=True, how='outer')

,key1,value,group_vale
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [171]:
left_df = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'], 'key2': [2000, 2001, 2002, 2001, 2002],
                        'data': np.arange(5)})

In [175]:
right_df = pd.DataFrame(np.arange(12).reshape((6, 2)), index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                        [2001, 2000, 2000, 2000, 2001, 2002]], columns=['event1', 'event2'])

In [176]:
left_df

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [177]:
right_df

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [179]:
pd.merge(left_df, right_df, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [181]:
pd.merge(left_df, right_df, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0
